In [7]:
from keras import applications, optimizers
from keras import backend as K
from keras import models as Models
from keras.models import Model 
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import os
import sys
import time

img_width, img_height = 256, 256
train_data_dir = "train_set"
validation_data_dir = "test_set"
batch_size = 32
epochs = 10

In [8]:
model = applications.VGG16(weights = "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5", include_top=False, input_shape = (img_width, img_height, 3))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [9]:
# Freeze layers
for layer in model.layers:
   layer.trainable = False

# Create output
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(10, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

print(model_final.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [10]:
# Initiate the train and test generators with data Augumentation 
train_datagen_rescale = ImageDataGenerator(rescale = 1./255)

train_datagen_aug = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen_rescale = ImageDataGenerator(rescale = 1./255)

test_datagen_aug = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen_aug.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen_aug.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

Found 1450 images belonging to 10 classes.
Found 264 images belonging to 10 classes.


In [11]:
# Train the model
time_start = time.time()

model_final.fit_generator(
    train_generator,
    epochs = epochs,
    validation_data = validation_generator)

time_elapsed = time.time() - time_start

# Record time used
print(time_elapsed)

Epoch 1/10
46/46 [==============================] - 25s 546ms/step - loss: 1.7426 - acc: 0.4122 - val_loss: 1.4605 - val_acc: 0.5833
Epoch 2/10
46/46 [==============================] - 24s 531ms/step - loss: 1.5137 - acc: 0.5231 - val_loss: 1.3186 - val_acc: 0.6212
Epoch 3/10
46/46 [==============================] - 20s 443ms/step - loss: 1.4185 - acc: 0.5599 - val_loss: 1.2513 - val_acc: 0.7045
Epoch 4/10
46/46 [==============================] - 24s 517ms/step - loss: 1.3507 - acc: 0.5933 - val_loss: 1.1867 - val_acc: 0.6780
Epoch 5/10
46/46 [==============================] - 24s 516ms/step - loss: 1.3043 - acc: 0.6128 - val_loss: 1.1132 - val_acc: 0.6629
Epoch 6/10
46/46 [==============================] - 20s 444ms/step - loss: 1.2235 - acc: 0.6360 - val_loss: 1.0668 - val_acc: 0.7083
Epoch 7/10
46/46 [==============================] - 20s 445ms/step - loss: 1.2160 - acc: 0.6325 - val_loss: 1.0331 - val_acc: 0.7008
Epoch 8/10
46/46 [==============================] - 25s 541ms/step - 

In [12]:
score = model_final.evaluate_generator(validation_generator)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.920671289617
Test accuracy: 0.727272727273
